In [1]:
import nltk
from snowballstemmer import TurkishStemmer
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import numpy
import random
import json


with open(r"C:\Users\user\Desktop\Zlik yap\data.json") as file:
    data = json.load(file)


stemmer=TurkishStemmer()
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [stemmer.stemWord(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stemWord(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

Using TensorFlow backend.


In [3]:
labels

['ayrılma', 'bilgi', 'learn', 'selamlama']

In [2]:
model = Sequential()
model.add(Dense(8,input_shape=(len(training[0]),),activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dense(4,activation="softmax"))
model.summary()
model.compile(Adam(lr=.001),loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(training, output,epochs=150, verbose=2,batch_size=4)

W0305 22:22:20.775223 12120 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\gputest\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0305 22:22:20.794087 12120 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\gputest\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0305 22:22:20.796081 12120 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\gputest\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0305 22:22:20.852512 12120 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\gputest\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0305 22:22:20.860455 12120 depr

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 256       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 68        
Total params: 468
Trainable params: 468
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
 - 2s - loss: 1.4023 - acc: 0.1176
Epoch 2/150
 - 0s - loss: 1.3909 - acc: 0.1765
Epoch 3/150
 - 0s - loss: 1.3834 - acc: 0.2941
Epoch 4/150
 - 0s - loss: 1.3755 - acc: 0.2941
Epoch 5/150
 - 0s - loss: 1.3676 - acc: 0.2941
Epoch 6/150
 - 0s - loss: 1.3602 - acc: 0.3529
Epoch 7/150
 - 0s - loss: 1.3526 - acc: 0.3529
Epoch 8/150
 - 0s - loss: 1.3458 - acc: 0.3529
Epoch 9/150
 - 0s - loss: 1.3380

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stemWord(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Chatbot ile konuşmaya başlayabilirsiniz (quit yazarak çıkabilirsiniz)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict(np.asanyarray([bag_of_words(inp, words)]))[0]
        print(results)
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.85:
            
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
        else:
            print("Tam olarak anlayamadım")

chat()

Chatbot ile konuşmaya başlayabilirsiniz (quit yazarak çıkabilirsiniz)!
You: Selam
[0.01160604 0.00361175 0.00262857 0.9821536 ]
Nasıl yardımcı olabilirim
You: afrika
[0.1957812  0.02803112 0.02615681 0.7500308 ]
Tam olarak anlayamadım
You: yapay zekadan bahseder misin
[0.00211809 0.82973456 0.07513086 0.09301645]
Tam olarak anlayamadım
You: makine öğrenmesi nedir
[0.00146018 0.9341736  0.0071442  0.05722203]
Matematiksel ve istatistiksel yöntemler kullanarak mevcut verilerden çıkarımlar yapmayı sağlayan bir metod
